In [21]:
import os
import sys
import traceback

import sys, os, time, argparse

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as nfunc
from torch.nn.parameter import Parameter

import random
import argparse
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.optim as optim
from torch.utils.data import DataLoader


try:
    import cupy as cp
except ImportError:
    cp = None

import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import Variable, optimizers, cuda, serializers
#import matplotlib.pyplot as plt

gpu = ""

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

# chainer code

In [15]:
import chainer
import chainer.functions as F
from chainer import Variable, optimizers, cuda, serializers
%load_ext autoreload
%autoreload 2


from ExpUtils import *

from chainer_func.source.CostFunc import *
from chainer_func.source.utils import mkdir_p
from chainer_func.models.cnn import CNN
from chainer_func.load_dataset import load_dataset


In [17]:


def train(args):
    train_l, train_ul, test = load_dataset(args.data_dir, valid=args.validation, dataset_seed=args.dataset_seed)
    wlog("N_train_labeled:{}, N_train_unlabeled:{}".format(train_l.N, train_ul.N))
    enc = CNN(n_outputs=args.n_categories, dropout_rate=args.dropout_rate, top_bn=args.top_bn)
    if args.gpu > -1:
        enc.to_gpu()

    optimizer = optimizers.Adam(alpha=args.lr, beta1=args.mom1)
    optimizer.setup(enc)
    optimizer.use_cleargrads()
    alpha_plan = [args.lr] * args.num_epochs
    beta1_plan = [args.mom1] * args.num_epochs
    for i in range(args.epoch_decay_start, args.num_epochs):
        alpha_plan[i] = float(args.num_epochs - i) / (args.num_epochs - args.epoch_decay_start) * args.lr
        # alpha_plan[i] = args.lr * 0.5 ** int((i - args.epoch_decay_start) / args.decay_step + 1)
        beta1_plan[i] = args.mom2

    accs_test = np.zeros(args.num_epochs)
    cl_losses = np.zeros(args.num_epochs)
    ul_losses = np.zeros(args.num_epochs)
    mkdir_p(args.log_dir)
    iter_no = 0
    start = time.time()
    for epoch in range(args.num_epochs):
        optimizer.alpha = alpha_plan[epoch]
        optimizer.beta1 = beta1_plan[epoch]
        sum_loss_l = 0
        sum_loss_ul = 0
        for it in range(args.num_iter_per_epoch):
            iter_no += 1
            # set_framework_seed(iter_no % 10000)
            with chainer.using_config("train", True):
                x, t = train_l.get(args.batchsize, gpu=args.gpu, aug_trans=args.aug_trans, aug_flip=args.aug_flip)
                loss_l = loss_labeled(enc, Variable(x), t, args)
                x_u, _ = train_ul.get(args.batchsize_ul, gpu=args.gpu, aug_trans=args.aug_trans, aug_flip=args.aug_flip)
                loss_ul = loss_unlabeled(enc, Variable(x_u), None, args, Variable(x), Variable(t))
                if args.method == "ce":
                    loss_total = loss_l
                else:
                    loss_total = loss_l + loss_ul
                enc.cleargrads()
                loss_total.backward()
                optimizer.update()
                sum_loss_l += loss_l.data
                sum_loss_ul += loss_ul.data
            end = time.time()
            cl_losses[epoch] = sum_loss_l / args.num_iter_per_epoch
            ul_losses[epoch] = sum_loss_ul / args.num_iter_per_epoch
        if (iter_no + 1) % args.eval_freq == 0:
            with chainer.using_config("train", False):
                acc_test_sum = 0
                test_x, test_t = test.get()
                N_test = test_x.shape[0]
                for i in range(0, N_test, args.batchsize_eval):
                    x = test_x[i:i + args.batchsize_eval]
                    t = test_t[i:i + args.batchsize_eval]
                    if args.gpu > -1:
                        x, t = cuda.to_gpu(x, device=args.gpu), cuda.to_gpu(t, device=args.gpu)
                    _, acc = loss_test(enc, Variable(x), Variable(t))
                    acc_test_sum += acc * x.shape[0]
                accs_test[epoch] = acc_test_sum / N_test
                wlog("Epoch:{}, iteration:{}, nll loss:{}, unlabeled loss:{}, time:{}".format(
                    epoch, iter_no, cl_losses[epoch], ul_losses[epoch], end - start))
                wlog("test acc:{}, error rate: {}".format(accs_test[epoch], 100 * (1-accs_test[epoch])))
                start = end

In [28]:
def parse_args():
    args = argparse.Namespace()
    args.dataset = "cifar10"
    args.trainer = "vat"
    args.lr = 0.001
    args.arch = "CNN9c"
    args.iterations = 1000
    args.seed = 1
    args.size = 100
    args.no_cuda = False
    
    args.xi = 1e-6
    args.eps = 10
    args.k = 1
    args.use_entmin = False
    args.alpha = 1
    args.mom1 = 0.9
    args.mom2 = 0.5
    args.reg_lamb = 1
    
    args.gpu = -1
    args.data_dir = "./dataset/cifar10/"
    args.log_dir = "log"
    args.n_categories = 10
    args.eval_freq = 5
    args.snapshot_freq = 20
    args.aug_flip = False
    args.aug_trans = False
    args.validation = False
    args.dataset_seed = 1
    args.batchsize = 32
    args.batchsize_ul = 128
    args.batchsize_eval = 100
    args.num_epochs = 120
    args.num_iter_per_epoch = 500
    args.epoch_decay_start = 0
    args.method = "vat"
    args.epsilon = 10
    args.extra_lamb = 1
    args.dropout_rate = 0.5
    args.top_bn = False
    args.vis = False
    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    args.data_dir = os.path.join("./dataset/%s" % args.dataset)

    chainer.global_config.cudnn_deterministic = True
    random.seed(args.seed)
    if int(args.gpu) > -1:
        chainer.cuda.get_device(args.gpu).use()
    np.random.seed(args.seed)
    if cp is not None:
        cp.random.seed(args.seed)
    return args

args = parse_args()
arg = parse_args()
arg.exp = "avg"
saver = ExpSaver("VATSemi-%s" % arg.method, arg, ["epsilon", "dropout_rate"], None)
if arg.vis:
    saver.init_writer(["epsilon", "dropout_rate"])
arg.log_dir = saver.log_dir
run_time = saver.log_dir
wlog("run time marker %s" % run_time)
wlog("args in this experiment %s", '\n'.join(str(e) for e in sorted(vars(arg).items())))
# noinspection PyBroadException

train(arg)


2019-05-09 21:19:56,212 - <ipython-input-28-142f980327d8>[line:65]: run time marker C:\Users\34639\project\results\VATSemi\cifar10\vat-epsilon=10-dropout_rate=0.5-20190509211955_running
2019-05-09 21:19:56,229 - <ipython-input-28-142f980327d8>[line:66]: args in this experiment ('alpha', 1)
('arch', 'CNN9c')
('aug_flip', False)
('aug_trans', False)
('batchsize', 32)
('batchsize_eval', 100)
('batchsize_ul', 128)
('data_dir', './dataset/cifar10')
('dataset', 'cifar10')
('dataset_seed', 1)
('dropout_rate', 0.5)
('epoch_decay_start', 0)
('eps', 10)
('epsilon', 10)
('eval_freq', 5)
('exp', 'avg')
('extra_lamb', 1)
('gpu', -1)
('iterations', 1000)
('k', 1)
('log_dir', 'C:\\Users\\34639\\project\\results\\VATSemi\\cifar10\\vat-epsilon=10-dropout_rate=0.5-20190509211955_running')
('lr', 0.001)
('method', 'vat')
('mom1', 0.9)
('mom2', 0.5)
('n_categories', 10)
('no_cuda', False)
('num_epochs', 120)
('num_iter_per_epoch', 500)
('reg_lamb', 1)
('seed', 1)
('size', 100)
('snapshot_freq', 20)
('task

Traceback (most recent call last):
  File "d:\software\anaconda2\envs\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-142f980327d8>", line 69, in <module>
    train(arg)
  File "<ipython-input-17-22a1ca30a285>", line 38, in train
    loss_ul = loss_unlabeled(enc, Variable(x_u), None, args, Variable(x), Variable(t))
  File "E:\project\other\DeepLearningCMP\chainer_func\source\CostFunc.py", line 55, in loss_unlabeled
    vat_loss = args.reg_lamb * loss.vat_loss(forward, loss.distance, x, epsilon=args.epsilon, xi=XI, p_logit=logit.data)
  File "E:\project\other\DeepLearningCMP\chainer_func\source\chainer_functions\loss.py", line 30, in vat_loss
    kl_loss.backward()
  File "d:\software\anaconda2\envs\venv\lib\site-packages\chainer\variable.py", line 980, in backward
    self._backward_main(retain_grad, loss_scale)
  File "d:\software\anaconda2\envs\venv\lib\site-packages\chain

KeyboardInterrupt: 